In [1]:
import torch
import numpy as np
from botorch.models import SingleTaskGP
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.fit import fit_gpytorch_model
from botorch.acquisition.multi_objective import qExpectedHypervolumeImprovement
from botorch.optim import optimize_acqf
from botorch.utils.multi_objective.box_decompositions import NondominatedPartitioning
from botorch.utils.multi_objective.pareto import is_non_dominated
from botorch.utils import standardize
from botorch.acquisition.multi_objective.objective import IdentityMCMultiOutputObjective
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [2]:
paramConfig = {
    'c1': {'lowerBound': 0, 'upperBound': 1, 'exponent': 1.0, 'name': 'W', 'unit': 'dimensionless', 'type': 'hardening'}, 
    'c2': {'lowerBound': 0, 'upperBound': 2, 'exponent': 1000.0, 'name': 'K', 'unit': 'MPa', 'type': 'yielding'}, 
    'c3': {'lowerBound': 0, 'upperBound': 1, 'exponent': 0.1, 'name': 'e0', 'unit': 'dimensionless', 'type': 'hardening'}, 
    'c4': {'lowerBound': 0, 'upperBound': 1, 'exponent': 1.0, 'name': 'n', 'unit': 'dimensionless', 'type': 'hardening'}, 
    'c5': {'lowerBound': 0, 'upperBound': 2, 'exponent': 1000.0, 'name': 'sigma_y', 'unit': 'MPa', 'type': 'yielding'}, 
    'c6': {'lowerBound': 0, 'upperBound': 1, 'exponent': 1000.0, 'name': 'sigma_sat', 'unit': 'MPa', 'type': 'hardening'}, 
    'c7': {'lowerBound': 0, 'upperBound': 1, 'exponent': 1000.0, 'name': 'b', 'unit': 'dimensionless', 'type': 'hardening'}
}

geometries = ['NDBR50', 'NDBR6', 'CHD6']

yieldingIndices = {'NDBR50': 200, 'NDBR6': 200, 'CHD6': 1200}

In [3]:
# Load the data
combined_interpolated_params_to_geoms_FD_Curves_smooth = np.load('combined_interpolated_param_to_geom_FD_Curves_smooth.npy', allow_pickle=True).tolist()
targetCurves = np.load('targetCurves.npy', allow_pickle=True).tolist()

# Define the function for the RMSE loss
def lossFD(targetDisp, targetForce, simForce):
    return torch.sqrt(torch.mean((simForce - targetForce)**2))

# Calculate losses and prepare data for model
params = []
losses = []

for param_tuple, geom_to_simCurves in combined_interpolated_params_to_geoms_FD_Curves_smooth.items():
    #print(param_tuple)
    params.append([value for param, value in param_tuple])
    # The minus sign is because BOTORCH tries to maximize objectives, but we want to minimize the loss
    loss_iter = []
    for geometry in geometries:
        yieldingIndex = yieldingIndices[geometry]
        loss_iter.append(- lossFD(
            torch.tensor(targetCurves[geometry]["displacement"][yieldingIndex:]), 
            torch.tensor(targetCurves[geometry]["force"][yieldingIndex:]), 
            torch.tensor(geom_to_simCurves[geometry]["force"][yieldingIndex:])
        ))
    losses.append(loss_iter)
    
# Convert your data to the tensor(float 64)
X = torch.tensor(params, dtype=torch.float64)
Y = torch.stack([torch.tensor(loss, dtype=torch.float64) for loss in losses])

# Normalize X to have range of [0, 1]
minmax_scaler = MinMaxScaler()
X_normalized = torch.tensor(minmax_scaler.fit_transform(X.numpy()), dtype=torch.float64)

# Standardize Y to have zero mean and unit variance
standard_scaler = StandardScaler()
Y_standardized = torch.tensor(standard_scaler.fit_transform(Y.numpy()), dtype=torch.float64)

# Define the bounds of the search space
lower_bounds = torch.tensor([paramConfig[param]['lowerBound'] * paramConfig[param]['exponent'] for param in paramConfig.keys()]).float()
upper_bounds = torch.tensor([paramConfig[param]['upperBound'] * paramConfig[param]['exponent'] for param in paramConfig.keys()]).float()


lower_bounds = minmax_scaler.transform(lower_bounds.reshape(1, -1)).squeeze()
upper_bounds = minmax_scaler.transform(upper_bounds.reshape(1, -1)).squeeze()

bounds = torch.tensor(np.array([lower_bounds, upper_bounds])).float()

# Initialize model
model = SingleTaskGP(X_normalized, Y_standardized)
mll = ExactMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_model(mll)

# **Reference Point**

# qEHVI requires specifying a reference point, which is the lower bound on the objectives used for computing hypervolume. 
# In this tutorial, we assume the reference point is known. In practice the reference point can be set 
# 1) using domain knowledge to be slightly worse than the lower bound of objective values, 
# where the lower bound is the minimum acceptable value of interest for each objective, or 
# 2) using a dynamic reference point selection strategy.

ref_point = Y_standardized.min(dim=0).values - 0.01

partitioning = NondominatedPartitioning(ref_point=ref_point, Y=Y_standardized)
acq_func = qExpectedHypervolumeImprovement(
    model=model,
    partitioning=partitioning,
    ref_point=ref_point,
    objective=IdentityMCMultiOutputObjective(),
)


# Optimize the acquisition function
candidates, _ = optimize_acqf(
    acq_function=acq_func,
    bounds=bounds,
    q=1, #q: This is the number of points to sample in each step. 
    num_restarts=10, # num_restarts: This is the number of starting points for the optimization.
    raw_samples=1000, # raw_samples: This is the number of samples to draw when initializing the optimization
)

# Unnormalize the candidates
candidates = minmax_scaler.inverse_transform(candidates.detach().numpy())

#converting to dictionary
next_param_dicts = [{param: value.item() for param, value in zip(paramConfig.keys(), next_param)} for next_param in candidates]

print(next_param_dicts)

[{'c1': 0.865545392036438, 'c2': 1796.846435546875, 'c3': 0.09999999403953552, 'c4': 0.06777941435575485, 'c5': 923.2322387695312, 'c6': 457.3926696777344, 'c7': 138.47036743164062}]
